In [1]:
import logging
from galvasr2.align.spark.align_lib import load_audio_id_text_id_mapping, load_transcripts


spark = SparkSession.builder \
                      .master("local[*]") \
                      .appName("NER") \
                      .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
                      .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
                      .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
                      .config("spark.driver.memory", "40g")\
                      .config("spark.executor.memory", "40g")\
                      .config("spark.rpc.askTimeout", "480s")\
                      .config("spark.executor.heartbeatInterval", "20000ms")\
                      .config("spark.eventLog.enabled", "true")\
                      .getOrCreate()
# spark.sparkContext.setLogLevel("INFO") # "ALL" for very verbose logging                                                                                                                                   
logging.getLogger("py4j").setLevel(logging.ERROR)
catalogue_df = load_audio_id_text_id_mapping(spark, "gs://the-peoples-speech-west-europe/archive_org/Mar_7_2021/CC_BY_SA_EXPANDED_LICENSES_FILTERED_ACCESS.jsonl.gz")
training_sample_rows = catalogue_df.collect()

# Comment this out to load everything. It might takes ~15 minute, in my experience, on an 8 core machine.
training_sample_rows = training_sample_rows[:100]
transcripts_df = load_transcripts(spark,
                                  "gs://the-peoples-speech-west-europe/archive_org/Mar_7_2021/CC_BY_SA_EXPANDED_LICENSES_FILTERED_ACCESS",
                                  training_sample_rows)
transcripts_pdf = transcripts_df.toPandas()

/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/install/miniconda3/envs/100k-hours-lingvo-3/lib/python3.7/importlib/_bootst

In [2]:
transcripts_pdf.count()

transcript          100
text_document_id    100
identifier          100
dtype: int64

In [3]:
transcripts_pdf

,transcript,text_document_id,identifier
0,good morning the Subcommittee on good morning ...,gov.house.ogr.fw.20100727.1.auto.srt,gov.house.ogr.fw.20100727.1
1,many will resume many will resume many will ...,gov.house.ogr.20090611.2.auto.srt,gov.house.ogr.20090611.2
2,OK. So home where. We. Are. Right. Here. First...,Jericho_Selectboard_11-20-14.asr.srt,Jericho_Selectboard_11-20-14
3,And welcome to this regularly scheduled meetin...,Holland_City_Council_-_May_1_2013.asr.srt,Holland_City_Council_-_May_1_2013
4,I'm very excited have all of you guys here we ...,Digital_Marketing_Summer_Sessions_-_Google_Ana...,Digital_Marketing_Summer_Sessions_-_Google_Ana...
...,...,...,...
95,Everybody miles Larsen with the city of Saraso...,Commission_Report_Night_5_16_16.asr.srt,cosflv-Commission_Report_Night_5_16_16
96,This is a story of a holiday and. Helping make...,DiningTo1951.asr.srt,DiningTo1951
97,Why keeping the car in the I thought. Yes Mr J...,Nonesuch1934.asr.srt,Nonesuch1934
98,Now a cigarette that some lowly and tar and ni...,MontclairCig.asr.srt,MontclairCig
